# SSD 300 model kvantizacia

Postup ako pri ResNet

Model je predtrénovaný na COCO datasete.
Model z:
https://pytorch.org/hub/nvidia_deeplearningexamples_ssd/
https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Detection/SSD

Definícia modelu prebratá z:
https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/Detection/SSD/src/model.py
a upravená.

Model upravíme pre potreby kvantizácie, úprava modelu podľa tutorialu v Distiller Framework.

## Loading pre-trained model

In [1]:
import torch

In [2]:
torch.cuda.empty_cache()
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)

Using cache found in /home/bohumil/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [3]:
ssd_model


SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [4]:
#loading pre-processing utils for model

utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

Using cache found in /home/bohumil/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Chrbtica tohto SSD modelu je ResNet, môžeme teda priamo použiť ResNet implementovaný v rámci Distiller

### Base precision

Potrebujeme si uložiť váhy tohto modelu, následne vyrobiť impl pre Distiler kvantizaciu a dayť váhy tomuto novému modelu.

In [5]:
import os
chckpt_name = "./ssd_model_ORIG.pth"
if os.path.isfile(chckpt_name):
    torch.save(ssd_model,chckpt_name)


/home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type SSD300. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Co

In [6]:
#ssd_model.to('cuda')
#ssd_model.eval()


In [7]:
import torch.utils
import torchvision


COCO_PATH = "/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017"
COCO_ANN = "/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/annotations/instances_val2017.json"
# 
# dataset = torchvision.datasets.CocoDetection(root= COCO_PATH,annFile= COCO_ANN, transform=utils.prepare_tensor)
# 
# #musime pouzit torch dataloader, lebo distiler nema coco impl
# subset_size = 1.0
# val_split = 0.8
# 
# batch_size_gpu = 32
# workers_gpu = 0
# 
# dataloader = torch.utils.data.DataLoader(dataset, batch_size= batch_size_gpu, num_workers=workers_gpu)



In [8]:
def get_images_list(dir):
    # f= []
    return [ os.path.join(dir,f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir,f))]


In [9]:
import os
images_list = get_images_list(COCO_PATH)
len(images_list)

5000

In [10]:
def prepare(images_list, prec):
    inputs = [utils.prepare_input(uri) for uri in images_list]
    tensor = utils.prepare_tensor(inputs, precision == prec)
    return inputs,tensor

In [11]:
#torch.cuda.empty_cache()
#part = images_list[:25]
#%time inputs, tensor = prepare(part, 'fp16')



In [12]:
# results_per_input = utils.decode_results(detections_batch)
# best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

In [13]:
# classes_to_labels = utils.get_coco_object_dictionary()

In [14]:
# from matplotlib import pyplot as plt
# import matplotlib.patches as patches
# 
# for image_idx in range(len(best_results_per_input)):
#     fig, ax = plt.subplots(1)
#     # Show original, denormalized image...
#     image = inputs[image_idx] / 2 + 0.5
#     ax.imshow(image)
#     # ...with detections
#     bboxes, classes, confidences = best_results_per_input[image_idx]
#     for idx in range(len(bboxes)):
#         left, bot, right, top = bboxes[idx]
#         x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
#         rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
#         ax.add_patch(rect)
#         ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
# plt.show()



In [15]:
print('ahoj')

ahoj


In [16]:
from src.model import SSD300 , ResNet

ssd_dist = SSD300()

In [17]:
ssd_dist

SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): DistillerBottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-0

In [18]:
# ssd_dist2 = SSD300(backbone=ResNet(backbone='resnet18'))

In [19]:
part = images_list[:10]
inputs, tensor = prepare(part, 'fp16')

In [20]:
part

['/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000297562.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000018837.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000512476.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000502136.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000150726.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000097679.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000052462.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000012280.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000110999.jpg',
 '/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco/images/val2017/000000385190.jpg']

In [21]:
# ssd_dist.to('cuda')
# ssd_dist.eval()

In [22]:
# eval(inputs, tensor, utils, ssd_dist)


In [23]:
# torch.cuda.empty_cache()
ssd_dist.to('cpu')

ssd_model.to('cuda')
ssd_model.eval()

SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [24]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)

In [25]:
results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

In [26]:
classes_to_labels = utils.get_coco_object_dictionary()

In [27]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(classes_to_labels, fp)

In [28]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches

for image_idx in range(len(best_results_per_input)):
    fig, ax = plt.subplots(1)
    # Show original, denormalized image...
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    # ...with detections
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        left, bot, right, top = bboxes[idx]
        x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], 
                                          confidences[idx]*100), bbox=dict(facecolor='white', 
                                                                           alpha=0.5))
plt.show()